In [1]:
# # Install all required packages for the project
# !pip install python-dotenv  
# !pip install dask  
# !pip install yfinance --upgrade
# !dnf install python-jinja2 python3-jinja2-cli 
# from rich.pretty import pprint

from IPython.display import Markdown, display
import warnings

warnings.filterwarnings("ignore")

# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
import os
from dotenv import load_dotenv

dotenv_path = '/x/1_projects/dsi/scaling_to_production/01_materials/labs/.env'
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

PRICE_DATA = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(PRICE_DATA, '*/*/*.parquet'))
parquet_files[:10]  # print the 10 first found files

['../../05_src/data/prices/WAB/WAB_2018.parquet/part.0.parquet',
 '../../05_src/data/prices/WAB/WAB_2012.parquet/part.0.parquet',
 '../../05_src/data/prices/WAB/WAB_2013.parquet/part.0.parquet',
 '../../05_src/data/prices/WAB/WAB_2021.parquet/part.0.parquet',
 '../../05_src/data/prices/WAB/WAB_2014.parquet/part.0.parquet',
 '../../05_src/data/prices/WAB/WAB_2005.parquet/part.0.parquet',
 '../../05_src/data/prices/WAB/WAB_2024.parquet/part.0.parquet',
 '../../05_src/data/prices/WAB/WAB_2001.parquet/part.0.parquet',
 '../../05_src/data/prices/WAB/WAB_2004.parquet/part.0.parquet',
 '../../05_src/data/prices/WAB/WAB_2016.parquet/part.0.parquet']

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
import dask.dataframe as dd
import numpy as np

dd_px = dd.read_parquet(parquet_files[:1]).set_index("ticker")
dd_px

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
npartitions=1,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,string,string,int32
,...,...,...,...,...,...,...,...,...,...


In [6]:
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1
).assign(
    hi_lo_range= lambda x: x['High']-x['Low']
))

dd_feat.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
WAB,2018-01-02,81.720001,83.110001,80.440002,82.930000,79.660133,760200,Industrials,Construction Machinery & Heavy Transportation ...,2018,NaN,NaN,2.669998
WAB,2018-01-03,82.620003,83.050003,81.650002,82.260002,79.016533,1030100,Industrials,Construction Machinery & Heavy Transportation ...,2018,82.930000,-0.008079,1.400002
WAB,2018-01-04,83.040001,84.010002,82.809998,83.370003,80.082787,673700,Industrials,Construction Machinery & Heavy Transportation ...,2018,82.260002,0.013494,1.200005
WAB,2018-01-05,83.680000,83.959999,80.690002,81.830002,78.603493,985600,Industrials,Construction Machinery & Heavy Transportation ...,2018,83.370003,-0.018472,3.269997
WAB,2018-01-08,81.830002,82.470001,81.669998,82.370003,79.122200,312800,Industrials,Construction Machinery & Heavy Transportation ...,2018,81.830002,0.006599,0.800003


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [7]:
df_feat_pandas=dd_feat.compute()

# Double checking the df_feat_pandas type
print(f"Type of `dd_feat`: {type(dd_feat)}")
print(f"Type of `df_feat_pandas`: {type(df_feat_pandas)}")

df_feat_pandas['rolling_avg_return'] = df_feat_pandas['returns'].rolling(10).mean()
df_feat_pandas.head()

Type of `dd_feat`: <class 'dask_expr._collection.DataFrame'>
Type of `df_feat_pandas`: <class 'pandas.core.frame.DataFrame'>


,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag_1,returns,hi_lo_range,rolling_avg_return
ticker,,,,,,,,,,,,,,
WAB,2018-01-02,81.720001,83.110001,80.440002,82.930000,79.660133,760200,Industrials,Construction Machinery & Heavy Transportation ...,2018,NaN,NaN,2.669998,NaN
WAB,2018-01-03,82.620003,83.050003,81.650002,82.260002,79.016533,1030100,Industrials,Construction Machinery & Heavy Transportation ...,2018,82.930000,-0.008079,1.400002,NaN
WAB,2018-01-04,83.040001,84.010002,82.809998,83.370003,80.082787,673700,Industrials,Construction Machinery & Heavy Transportation ...,2018,82.260002,0.013494,1.200005,NaN
WAB,2018-01-05,83.680000,83.959999,80.690002,81.830002,78.603493,985600,Industrials,Construction Machinery & Heavy Transportation ...,2018,83.370003,-0.018472,3.269997,NaN
WAB,2018-01-08,81.830002,82.470001,81.669998,82.370003,79.122200,312800,Industrials,Construction Machinery & Heavy Transportation ...,2018,81.830002,0.006599,0.800003,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Not necessarily, Dask can handle this operation internally, enabling more efficient calculations and utilization of machine resources.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.